In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Crop_recommendation.csv')

In [3]:
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [4]:
data.isnull().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [5]:
data['label'].value_counts()

label
rice           100
maize          100
chickpea       100
kidneybeans    100
pigeonpeas     100
mothbeans      100
mungbean       100
blackgram      100
lentil         100
pomegranate    100
banana         100
mango          100
grapes         100
watermelon     100
muskmelon      100
apple          100
orange         100
papaya         100
coconut        100
cotton         100
jute           100
coffee         100
Name: count, dtype: int64

In [6]:
x = data.iloc[:, :-1]


In [7]:
from sklearn.preprocessing import StandardScaler
# Normalize the features
scaler = StandardScaler()
x = scaler.fit_transform(x)
x= pd.DataFrame(x)

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# Separate features and target
X = data.drop(columns=['label'])
y = data['label']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert target to one-hot encoding
y_one_hot = pd.get_dummies(y).values

In [18]:
import joblib
joblib.dump(label_encoder, 'label_encoder.joblib')

['label_encoder.joblib']

In [9]:
# Train-test split with stratification
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x, y_one_hot, test_size=0.2, stratify=y_one_hot, random_state=42
)

In [10]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
#Creating the model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

d:\projects\Crop Prediction\crop\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.0829 - loss: 3.0161 - val_accuracy: 0.2358 - val_loss: 2.6366
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3063 - loss: 2.5850 - val_accuracy: 0.5256 - val_loss: 2.0786
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5554 - loss: 2.0373 - val_accuracy: 0.6875 - val_loss: 1.4714
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7334 - loss: 1.3851 - val_accuracy: 0.7926 - val_loss: 0.9649
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8402 - loss: 0.9029 - val_accuracy: 0.8466 - val_loss: 0.6470
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9032 - loss: 0.5915 - val_accuracy: 0.8807 - val_loss: 0.4965
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9226 - loss: 0.4451 - val_accuracy: 0.8892 - val_loss: 0.3940
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9300 - loss: 0.3511 - val_accuracy: 0.9006 - val_loss

In [13]:
import joblib
joblib.dump(model, 'model.joblib')

['model.joblib']

In [67]:
model= joblib.load(open('model.joblib', 'rb'))

In [68]:
import numpy as np
sample_input = np.array([[90, 42, 43, 20.88, 82.00, 6.5, 203]])  # Example input
sample_input_scaled = scaler.transform(sample_input)
prediction = model.predict(sample_input_scaled)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])

d:\projects\Crop Prediction\crop\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step


In [71]:
np.argmax(prediction)

np.int64(20)

In [69]:
print(f"Recommended Crop: {predicted_label[0]}")

Recommended Crop: rice


In [72]:
#  Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.99


In [73]:
from sklearn.metrics import classification_report
import numpy as np
# Get predictions for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_test_classes = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Generate classification report
report = classification_report(y_test_classes, y_pred_classes, target_names=Encoder.categories_)
print("Classification Report:\n", report)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


ValueError: Number of classes, 22, does not match size of target_names, 1. Try specifying the labels parameter

In [43]:
Encoder.categories_[0]

array(['apple', 'banana', 'blackgram', 'chickpea', 'coconut', 'coffee',
       'cotton', 'grapes', 'jute', 'kidneybeans', 'lentil', 'maize',
       'mango', 'mothbeans', 'mungbean', 'muskmelon', 'orange', 'papaya',
       'pigeonpeas', 'pomegranate', 'rice', 'watermelon'], dtype=object)